# Nine men's morris Game Definition
This file contains the definiton of the game nine men's morris (nmm).

The starting state of nmm is defined as following. It contains multiple tuples:

1. The first tuple contains the number of stones white and black are still allowed to place.
2. The second tuple contains the state of the board. Every nine-tuple represents a ring of the possible positions of a stone starting at the top left corner continuing clock-wise. The positions can ether by empty `' '`, occupied by white `'w'` or occupied by black `'b'`.

In [ ]:
%run ./nmm-game-utils.ipynb

## Player Phase

In [ ]:
# Returns the phase of the given player
#   1. The player has to place his stones
#   2. The player is only allowed to move the stones along the lines
#   3. The player is allowed to jump with his stones
def playerPhase(s, p):
    # If the player has still stones left to place, he is still in phase 1
    if hasPlaceableStones(s, p):
        return 1
    # If the player is allowed to jump with his stones, he is in the last phase, phase 3
    elif isAllowedToJump(s, p):
        return 3
    # Else he is in phase 2, where he can only move his stones
    else:
        return 2

## Next States

In [ ]:
def nextStatesPhaseOne(s, p):
    # Extract the count of the stones and the board
    ((cw, cb), board) = s
    # Calculate all current mills the player has
    mills = findMills(board, p)

    # Place a stone in any empty cell
    placeBoards = {
        place(board, (r, c), p)
        for (r, c) in findEmptyCells(board)
    }
    # Calculate how many new mills were created
    boardMills = {
        board: countNewMills(board, mills, p)
        for board in placeBoards
    }

    # Here all final boards will be collected
    boards = {
        result
        for (b, count) in boardMills.items() 
        for result in ({b} if (count == 0) else executeMill(b, p))
    }

    # Remove one stone from the players stache
    (cw, cb) = (cw-1, cb) if p == 'w' else (cw, cb-1)

    # Return all possible states
    return { ((cw, cb), board) for board in boards }

In [ ]:
def nextStatesPhaseTwo(s, p):
    # Extract the count of the stones and the board
    ((cw, cb), board) = s
    # Calculate all current mills the player has
    mills = findMills(board, p)

    # Pick up one stone (so it can be placed again) and store coordinates of pickup cell
    removeBoards = {
        (place(board, (r, c), ' '), (r, c))
        for (r, c) in findCellsOf(board, p)
    }

    # Place the stone again at a neighboring cell
    placeBoards = {
        place(b, (r, c), p)
        for (b, rootCoords) in removeBoards
        for (r, c) in findNeighboringEmptyCells(b, rootCoords)
    }
    # Remove the old board, as at least one stone has to be moved
    placeBoards - { board }

    # Calculate how many new mills were created
    boardMills = {
        b: countNewMills(b, mills, p)
        for b in placeBoards
    }

    # Here all final boards will be collected
    boards = {
        result
        for (b, count) in boardMills.items() 
        for result in ({b} if (count == 0) else executeMill(b, p))
    }

    return { ((cw, cb), board) for board in boards }

In [ ]:
def nextStatesPhaseThree(s, p):
    # Extract the count of the stones and the board
    ((cw, cb), board) = s
    # Calculate all current mills the player has
    mills = findMills(board, p)

    # Pick up one stone (so it can be placed again)
    removeBoards = {
        place(board, (r, c), ' ')
        for (r, c) in findCellsOf(board, p)
    }

    # Place the stone again
    placeBoards = {
        place(b, (r, c), p)
        for b in removeBoards
        for (r, c) in findEmptyCells(b)
    }
    # Remove the old board, as at least one stone has to be moved
    placeBoards - { board }

    # Calculate how many new mills were created
    boardMills = {
        b: countNewMills(b, mills, p)
        for b in placeBoards
    }

    # Here all final boards will be collected
    boards = {
        result
        for (b, count) in boardMills.items() 
        for result in ({b} if (count == 0) else executeMill(b, p))
    }

    return { ((cw, cb), board) for board in boards }

In [ ]:
def nextStates(s, p):
    phase = playerPhase(s, p)
    if phase == 1:
        return nextStatesPhaseOne(s, p)
    elif phase == 2:
        return nextStatesPhaseTwo(s, p)
    else:
        return nextStatesPhaseThree(s, p)

## Game Progess

In [ ]:
def finished(s, p):
    return not hasEnoughStones(s, 'w') or \
           not hasEnoughStones(s, 'b') or \
           (p == 'w' and len(nextStates(s, 'w')) == 0) or \
           (p == 'b' and len(nextStates(s, 'b')) == 0)

Die Funktion `utility(s, p)` nimmt als Parameter einen Zustand `s`, für den gilt `finished(s) = true`, und einen Spieler `p`. Das Resultat ist ein Element der Menge `{-1, 0, 1}` und ist wie folgt zu interpretieren:
 * `-1`: Der Spieler `p` hat bei gegebenem Zustand `s` verloren.
 * `0`: Weder Spieler `p` noch sein Gegner haben gewonnen, es handlet sich um ein Unentschieden.
 * `1`: Der Spieler `p` hat bei gegebenem Zustand `s` gewonnen.

In [ ]:
def utility(s, p):
    utility  = -1 if not hasEnoughStones(s, p) else 0
    utility +=  1 if not hasEnoughStones(s, opponent(p)) else 0
    if utility != 0:
        return utility
    utility  = -1 if len(nextStates(s, p)) == 0 else 0
    utility +=  1 if len(nextStates(s, opponent(p))) == 0 else 0
    return utility